# Introduction to Jupyter

This assignment is run in a Jupyter notebook. Jupyter is a programming environment for python that combines sections of code (so called <i>cells</i>) with cells containing HTML text, just like regular websites. The nice thing about this is that it actually is an interactive notebook in which you can run python code, making the code easier to understand. 

Before the assignment starts, we will tell you the basics of jupyter which you need to know to do this assignment. For a more extensive list, you can visit [this website](https://towardsdatascience.com/jypyter-notebook-shortcuts-bf0101a98330).

## Navigating the notebook
<ul>
    <li> You can select a cell by single clicking it. You can see that the cell is selected by the <font color='blue'>blue line</font> on the left. <img src="example_selectedv2.png" alt="Example showing when a cell is selected.">
    <li> Once a cell is selected, you can navigate up and down using the arrow keys <b>🠕</b> and <b>🠗</b> respectively.
    <li> To run a selected cell, press the <b>Run</b> <img src="run_example.png" alt="Example showing the run button."> button on the top or use <b>Shift + Enter</b>. This will automatically select the next cell.
    <li> In case something goes wrong, you can restart the kernel with <b>↻</b> or go to <b> Kernel -> Restart </b>. It is also possible to restart and clear the output (deleting all variables) or restart and run all (automatically runs the whole notebook). 
</ul>

## Editing the cells
<ul>
    <li> A code cell can be edited by single pressing on the place you want to place your cursor. Note how the bar on the left is <font color='green'> green </font>
    <li> A text or markdown cell can be edited by double clicking it. This is also indicated in throughout the notebook and you will use this to answer the questions. The required answers are only textual so no special commands are needed. However, if you want to use more advanced formatting, you can search the 'desired formatting option + HTML' to find the corresponding HTML command for it.  
</ul>

# Computational Methods for Energy Networks: Assignments

In the first, lecture you have learned why we need to model networks. In the coming six assignments, we will get started with our first simulation of an energy network. 

To model a network and to simulate what is going on, we make use of programming languages and software packages. Programming languages are used in computer programming to implement algorithms. A wide scala of such languages exist, think of C++, Java, MATLAB, and python. In this week, we will work in python. Do not worry: you do not need to know this programming language. We will walk you through the examples step-by-step. 

A nice thing about popular programming languages is that you do not have to invent the wheel yourself all the time. Many software libraries exist that can do what you want. This way, many applications become accessible for you to use for your first analyses. <br>
In this week, we make use of two software libraries: [`pandapower`](https://pandapower.readthedocs.io/en/develop/#) and [`pandapipes`](https://www.pandapipes.org/)). pandapower is a well known library to simulate the flow of electrical power in a network, while pandapipes is the equivalent used to simulate gas propagation in a network. 

# Pandapower basics

In the first session, you will learn how to install pandapower and we will show you how a basic power system simulation looks like. For the MOOC, you can stick to this online environment. If you are interested to continue working yourself or if you want to compare the performance with your local machine, you can following the [download instructions](https://www.pandapower.org/start/).

## Importing the needed packages

In python, you have to import the packages you want to use. In this session, we will use the following packages:

In [ ]:
# import the desired packages
import pandapower
import pandapower.networks
import pandapower.topology
import pandapower.plotting
import pandapower.converter
import pandapower.estimation
import pandapower as pp
import copy

## Creating your first network

We start by creating to the following electricity network:


<img src="fa1_example_networkv3.png" alt="Simple electricity network contraining three buses, one connected to the grid, one connected to a generator and the last connected to a load. The three buses are connected in a line with two cables of 200 and 100 meters respectively.">

Remember from the video that a network consists of buses, representing consumers or producers, and branches, corresponding to electricity cables. We calculate the flow of electrical power in the cables.

We start by creating an empty network: 

In [ ]:
#create empty net
net = pp.create_empty_network() 

We have now created an empty network. You can look at it as a blank paper on which we are going to draw the buses and branches now. The variable `net` will now contain all information we provide on our electrical network.

Now let us add the 3 buses. We assign different voltage levels to the buses, namely one bus of 20 kV and two buses of 400 V. 

In [ ]:
#create buses
b1 = pp.create_bus(net, vn_kv=20., name="Bus 1")
b2 = pp.create_bus(net, vn_kv=0.4, name="Bus 2")
b3 = pp.create_bus(net, vn_kv=0.4, name="Bus 3")

We have a network with three buses inside variable `net`. But now we have to assign whether a bus is a consumer or a producer (generator). 

The first bus is going to provide power. In this case, this is not done by a generator/power plant, but by a connection to a grid, allowing for an infinite amount of power (from the perspective of the network). To create the grid connection, we use the command [`pp.create_ext_grid`](https://pandapower.readthedocs.io/en/v2.0.1/elements/ext_grid.html#create-function). For the grid, we also enter the bus voltage and its name.

We make the third bus a consumer, also called a load bus. Here power leaves the grid to, for example, supply a neighbourhood. This is done with the function [`pp.create_load`](https://pandapower.readthedocs.io/en/v2.0.1/elements/load.html#create-function). We specify to which bus the load is connected, its active and reactive power consumption and give it a name.

In [ ]:
#create bus elements
pp.create_ext_grid(net, bus=b1, vm_pu=1.02, name="Grid Connection")
pp.create_load(net, bus=b3, p_mw=0.1, q_mvar=0.05, name="Load");

Until now, we have three buses, a source and a load. However, the power cannot be transformed without transmission lines. We connect the three buses using two power cables, called lines, using the function [`pp.create_line`](https://pandapower.readthedocs.io/en/v2.0.1/elements/line.html#create-function). We specify which buses the lines connect, how long they are, and which [type](https://pandapower.readthedocs.io/en/v2.6.0/std_types/basic.html#lines) they are (to calculate the resistance of the lines towards electrical power). 

In [ ]:
# create branch elements
pp.create_line(net, from_bus=b1, to_bus=b2, length_km=0.2, name="Line 1", std_type="NAYY 4x150 SE") 
pp.create_line(net, from_bus=b2, to_bus=b3, length_km=0.1, name="Line 2", std_type="NAYY 4x150 SE");

We have now created the simple three bus, two cable network. We can doublecheck our work by running the `net` statement:

In [ ]:
net

If you want to know in more detail what kind of buses we created, you can run the [`net.bus`](https://pandapower.readthedocs.io/en/v2.6.0/elements/bus.html#input-parameters) command.
It gives the bus names, the voltage levels and whether they are in service. <b>Note how the index of Bus 1 is 0.</b>

In [ ]:
net.bus 

For the lines, we use [`net.line`](https://pandapower.readthedocs.io/en/v2.0.1/elements/line.html), which gives to what buses they are connected, how long they are and the resistance and reactance in ohm per km. 

In [ ]:
net.line

To simulate the power flow in the system, we have to make a call to a power flow function [`pp.runpp()`](https://pandapower.readthedocs.io/en/v2.6.0/powerflow/ac.html#balanced-ac-power-flow). Inside the brackets, we specify the network upon which we want to run the power flow computation.

In [ ]:
pp.runpp(net)

The [`runpp`](https://pandapower.readthedocs.io/en/v2.6.0/powerflow/ac.html#balanced-ac-power-flow) function stores the results inside the variable `net`. To see how much power is flowing from one bus to another, we have to display that information using [`net.res_line`](https://pandapower.readthedocs.io/en/v2.6.0/elements/line.html#result-parameters). The rows represent the two power cables. The information is stored in the columns. For more information about their meaning, visit the [pandapower website](https://pandapower.readthedocs.io/en/v2.6.0/elements/line.html#result-parameters).

In [ ]:
# show the power flow and losses in the lines
net.res_line.iloc[:,0:6]

The results show the amount of active and reactive power flowing from one bus to the other.
The first row, row 0, shows the power flow on the first cable. The first value in this row, `p_from_mw`, shows how much active power in MW is flowing from the first bus to the second bus. The third column shows how much power is flowing from the second bus to the first bus. As you can see, this is a negative value, meaning that power is flowing in the opposite direction. It actually displays how much power reaches the second bus. You see the slight discrepancy between the first and third column, which is the power loss in MW due to resistance of the cable. This is also displayed in column 5 (`pl_mw`). 



In the next session we will analyse the results of the power flow simulation in more detail. We will use plots to visualize the results in a better way. 

## <font color = 'blue'> Task 1:

<font color = 'blue'> Try to find the value of `pl_mw` of line 2. 

<font color='blue'> DOUBLE CLICK TO ANSWER; PRESS RUN WHEN DONE

## <font color = 'blue'> [OPTIONAL] Task 2

<font color = 'blue'> Expand the network to the one shown in the figure below by using the already provided lines of code.
<img src="expanded_networkv3.png" alt="The simple network is expanded to a 6 bus network.">
    
<font color = 'blue'> Hint: To do this, add the orange region to the existing network. 
Add two more load buses (one 400 V and one 700 V) and one more generator bus (20 kV) to the network. A generator can be added with [`pp.create_gen()`](https://pandapower.readthedocs.io/en/v2.6.0/elements/gen.html#create-function). The new loads are identical to the load at bus 3, except that load 6 has half the power consumption ([`p_mw`](https://pandapower.readthedocs.io/en/v2.6.0/elements/load.html#create-function)). The generator has a power output [`p_mw`](https://pandapower.readthedocs.io/en/v2.6.0/elements/gen.html#create-function) of 0.2 MW and a voltage [`vm_pu`](https://pandapower.readthedocs.io/en/v2.6.0/elements/gen.html#create-function) of 1.0 pu. Do not forget to connect the new buses with cables. Run the power flow again by executing the [`runpp`](https://pandapower.readthedocs.io/en/v2.6.0/powerflow/ac.html#balanced-ac-power-flow) command.


In [ ]:
# creator functions
bx = pp.create_bus(net, vn_kv=, name="")
pp.create_gen(net, bus=, p_mw=, vm_pu=)
pp.create_load(net, bus=, p_mw=, q_mvar=, name="");
pp.create_line(net, from_bus=, to_bus=, length_km=, name="", std_type="NAYY 4x150 SE") 

<font color='blue'> What is the value of `pl_mw` in the line between bus 2 and 3? Note: the line index and name can differ depending on how you created the lines.

<font color='blue'> DOUBLE CLICK TO ANSWER; PRESS RUN WHEN DONE

## <font color = 'blue'> [OPTIONAL] Task 3 

<font color = 'blue'> In this task, you will improve the network. You might have noticed from the previous task that the losses and cable loading are very high for several lines. This is caused by the way generators are modelled. Generators are implemented as PV buses, meaning their active power `p_mw` and voltage `vm_pu` setpoints are enforced. The generator at bus 4 has a voltage of 1.0 pu while the external grid to which it is closeby is 1.02 pu. The large voltage drop across the line results in a huge reactive power flow in the line which also causes active power losses. In short, the current configuration is sub-optimal.

Try to fix this issue by testing the following two options:
<ol> 
<li> Change the voltage setpoint `vm_pu` to 1.02 to match that of the external grid. You can use `net.gen.at[x, 'vm_pu'] = 1.02` to access the voltage setpoint of the x-th generator. Use the `net.gen` command to find the corresponding index; if Task 2 was done correctly, this should be 0.
<li> Remove the line between bus 1 and bus 4. Use `net.line` and `net.bus` to find the index of the desired line. Note that the indeces can differ depending on the order in which you created the lines. Once you have found the corresponding line, use `pp.drop_lines(net, [y])` where y is the index of the line between bus 1 and bus 4. </li>
</ol>

Hint: First copy your network using `network_optionx = copy.deepcopy(net)` before evaluating each option. This way you do not have to restore the previous actions. Don't forget to change `net` to `network_optionx` in the code above.

### <font color='blue'> Option 1

In [ ]:
# copy the network
network_option1 = copy.deepcopy(net)

In [ ]:
# inspect the gen object
network_option1.gen

In [ ]:
# change the setpoint
network_option1.gen.at[add_number_here, 'vm_pu'] = 1.02

In [ ]:
# run the simulation and get the results
pp.runpp(network_option1)
network_option1.res_line

### <font color='blue'> Option 2

In [ ]:
# copy the network
network_option2 = copy.deepcopy(net)

In [ ]:
# inspect the bus mappings
network_option2.bus

In [ ]:
# inspect the lines
network_option2.line

In [ ]:
# drop the corresponding line
pp.drop_lines(network_option2, [add_linenumber_here])

In [ ]:
# run the simulation and get the results
pp.runpp(network_option1)
network_option1.res_line

<font color='blue'> Which option is better (i.e. does lead to fewer losses and realistic cable loadings)? 

<font color='blue'> DOUBLE CLICK TO ANSWER; PRESS RUN WHEN DONE